# Colab GPU Training (visuomotor-robustness-study)

Steps to run:
1. In Colab: Runtime ? Change runtime type ? Hardware accelerator: GPU.
2. Get this folder into /content (upload a zip to Drive and unzip, or git clone).
3. Set REPO_DIR and DATA_FILE below (make sure the .pkl is inside REPO_DIR).
4. Run installs, then training, then evaluation.


In [ ]:
# Check GPU availability
!nvidia-smi

Fri Nov 28 03:49:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
/content/drive/MyDrive/visuomotor-robustness-study


Mounted at /content/drive


In [ ]:
# Install dependencies (run once per session)
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.5-cp312-cp312-linux_x86_64.whl size=99861623 sha256=421a15dfd0a2f1ec40d8a209d7c473a79300402cd69560dbc45d60b45c3d9131
  Stored in directory: /root/.cache/pip/wheels/db/04/ec/882b6dd5dcc790af09eb7b062a3b6205d43072e60cfec8a776
Successfully built pybullet


In [ ]:
! python data_collection.py --episodes 100 --output demonstrations_rgb.pkl --append

pybullet build time: Nov 28 2025 04:16:12
Appending to existing dataset: loaded 877 trajectories from demonstrations_rgb.pkl
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Note: rgb_array rendering is slow (~50-100ms/frame). This will take 15-30 minutes for 100 episodes.
Progress will be shown after each episode completes.

Episode 1/100: [OK] (36 steps, 5.5s/ep, ETA: 9m3s, Success rate: 100.0%)
Episode 2/100: [OK] (33 steps, 4.6s/ep, ETA: 7m32s, Success rate: 100.0%)
Episode 3/100: [OK] (47 steps, 4.9s/ep, ETA: 7m53s, Success rate: 100.0%)
Episode 4/100: [OK] (37 steps, 5.0s/ep, ETA: 8m3s, Success rate: 100.0%)
Episode 5/100: [OK] (37 steps, 4.9s/ep, ETA: 7m44s, Success rate: 100.0%)
Episode 6/100: [OK] (42 steps, 4.9s/ep, ETA: 7m43s, Success rate: 100.0%)
Episode 7/100: [OK] (37 steps, 5.0s/ep, ETA: 7m42s, Success rate: 100.0%)
Episode 8/100: [OK] (37 steps, 4.9s/ep, ETA: 7m28s,

In [ ]:
REPO_DIR = '/content/drive/MyDrive/visuomotor-robustness-study'
import os
os.chdir(REPO_DIR)
!python train.py --regime clean --backbone resnet --data demonstrations_rgb.pkl --epochs 50 --batch_size 64 --device cuda --loss_log logs/loss_clean_resnet_50ep.csv --val_split 0.1

Using device: cuda
Training with clean visual regime and resnet backbone...
Loaded 977 trajectories
Total (image, action) pairs: 33772
Dataset split -> train: 30395, val: 3377
Epoch 1/50, Train Loss: 0.173085, Val Loss: 0.240181
Epoch 2/50, Train Loss: 0.119464, Val Loss: 0.105248
Epoch 3/50, Train Loss: 0.105060, Val Loss: 0.111725
Epoch 4/50, Train Loss: 0.095297, Val Loss: 0.117233
Epoch 5/50, Train Loss: 0.087818, Val Loss: 0.099348
Epoch 6/50, Train Loss: 0.084922, Val Loss: 0.088035
Epoch 7/50, Train Loss: 0.078627, Val Loss: 0.083050
Epoch 8/50, Train Loss: 0.075354, Val Loss: 0.078450
Epoch 9/50, Train Loss: 0.070086, Val Loss: 0.075277
Epoch 10/50, Train Loss: 0.068548, Val Loss: 0.079736
Epoch 11/50, Train Loss: 0.068330, Val Loss: 0.077162
Epoch 12/50, Train Loss: 0.063390, Val Loss: 0.075108
Epoch 13/50, Train Loss: 0.061680, Val Loss: 0.071446
Epoch 14/50, Train Loss: 0.057024, Val Loss: 0.083622
Epoch 15/50, Train Loss: 0.056114, Val Loss: 0.070395
Epoch 16/50, Train Loss

In [ ]:
!python evaluate.py --policy models/policy_clean_resnet.pth --corruption none --episodes 50 --device cuda

pybullet build time: Nov 28 2025 04:16:12
Using device: cuda
Loading policy from models/policy_clean_resnet.pth...
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886

Evaluating policy for 50 episodes...
Episode 10/50, Success Rate: 30.00%
Episode 20/50, Success Rate: 15.00%
Episode 30/50, Success Rate: 16.67%
Episode 40/50, Success Rate: 22.50%
Episode 50/50, Success Rate: 24.00%

Evaluation Results:
  Policy: models/policy_clean_resnet.pth
  Corruption Type: None
  Episodes: 50
  Task Success Rate: 24.00% (12/50)
  Average Reward: -164.6600 (std: 66.2156)
  Final object→target distance (avg): 0.1541 m
  Final gripper→object distance (avg): 0.1321 m
